## Prerequisitos 
### 1 CUDA Toolkit 12.6

In [ ]:
!winget install --id=Nvidia.CUDA -e --version 12.6`

### 2 cudNN 9.1
**Descarga manual**

### 3 Pytorch (2.5GB)
<small>Nota: El ! al inicio indica que es un comando de shell.</small>

In [1]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu126

^C


### 4 Python packages
yolov11 -> pip
tensorRT -> instalarlo utilizando documentación de NVIDIA
numpy -> pip < 2.0
onnx 

In [ ]:
!pip3 install ultralytics onnx  "numpy<2.0"

### 5 Crear folders para datasets

In [ ]:
import os

parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
datasets_path = os.path.join(parent_dir, "datasets")

if not os.path.exists(datasets_path):
    folders = [
        "datasets/images/train",
        "datasets/images/val",
        "datasets/images/test",
        "datasets/labels/train",
        "datasets/labels/val",
        "datasets/labels/test",
    ]

    for folder in folders:
        os.makedirs(os.path.join(parent_dir, folder), exist_ok=True)


### 6 Crear directorios para segmentar imágenes y archivos: .yaml y classes.txt

In [3]:
import os
from os import getcwd, pardir, mkdir
from os.path import abspath, join, exists

# Obtener el directorio padre
parent_dir = abspath(join(getcwd(), pardir))

# Definir la ruta del directorio datasets en el directorio padre
dataset_dir = join(parent_dir, "datasets")
dataset_yaml_path = join(dataset_dir, "dataset.yaml")
model_dir = join(parent_dir, "model")
checkpoint_dir = join(parent_dir, "checkpoint")
visualization_dir = join(parent_dir, "visualization")

classes_name = []
classes_name_file = join(parent_dir, "model", "classes_name.txt")
with open(classes_name_file, "r", encoding="utf-8") as file:
    for linea in file:
        current_line = linea.strip()  # `.strip()` elimina espacios y saltos de línea
        classes_name.append(current_line)
        current_folder = join(parent_dir, linea)
        if not exists(current_line):
            mkdir(current_line)
    print(classes_name)

# Contenido del archivo YAML con rutas relativas desde el directorio padre
dataset_yaml_content = f"""\
# dataset.yaml

path: {dataset_dir}  # Ruta base del dataset
train: {join(dataset_dir, "images\\train")}  # Directorio con imágenes de entrenamiento
val: {join(dataset_dir, "images\\val")}  # Directorio con imágenes de validación
test: {join(dataset_dir, "images\\test")}  # Directorio con imágenes de prueba (opcional)

# Clases
nc: {len(classes_name)}  # Número de clases (modifica según tu dataset)
names: [{classes_name}]  # Nombres de las clases (ajusta según tus etiquetas)

batch_size: 16 # Tamaño del lote
epochs: 100 # Número de épocas de entrenamiento
learning_rate: 0.001 # Tasa de aprendizaje inicial
loss_function: yolov1_loss # Función de pérdida
optimizer: adam # Optimizador
metrics: [precision, recall, f1_score] # Métricas de evaluación

# Arquitectura del modelo
architecture:
  input_size: [448, 448]
  backbone: resnet50
  num_classes: 20

# Hiperparámetros de YOLOv11
yolov11:
  grid_size: [7, 7]
  boxes_per_cell: 2
  lambda_coord: 5
  lambda_noobj: 0.5

use_cuda: True # Entrenamiento en GPU
save_checkpoints: True # Guardar checkpoints
checkpoint_dir: {checkpoint_dir} # Ruta para guardar checkpoints
visualize: True # Visualización
visualization_dir: {visualization_dir} # Ruta para guardar visualizaciones






"""

# Crear el directorio si no existe
os.makedirs(dataset_dir, exist_ok=True)

# Crear el archivo dataset.yaml con el contenido
with open(dataset_yaml_path, "w") as file:
    file.write(dataset_yaml_content)

['aws_perform_exposure_2_calibrate', 'ff_gen_tools_generator_exposure', 'ff_gen_tools_generator_prep', 'ff_gen_tools_generator_release', 'ff_gen_tools_generator_standby', 'fpd_calibration_pass', 'fpd_offline', 'fpd_calibrating', 'generator_blocked', 'generator_exposure', 'generator_standby', 'generator_offline']


### 7. iterar por cada una de las carpetas
#### 7.1 leer el archivo coords.txt
contiene coordenadas relativas para YOLO
#### 7.2 buscar el indice correspondiente del nombre del directorio en classes.txt
es la primer columna del siguiente archivo
#### 7.3 crear los archivo .txt con los mismos nombres que los archivos de ese directorio
cada archivo tiene el indice seguido de las coordenadas ejemplo

`0 0.456250 0.230313 0.172500 0.025625`


In [ ]:
import os

from os.path import isdir, join, abspath

# Obtener el directorio padre
parent_dir = abspath(join(os.getcwd(), os.pardir))

# Definir la ruta del directorio models en el directorio padre
model_dir = join(parent_dir, "model")

classes_name = []
classes_name_file = join(model_dir, "classes_name.txt")
with open(classes_name_file, "r", encoding="utf-8") as file:
    for linea in file:
        current_line = linea.strip()  # `.strip()` elimina espacios y saltos de línea
        classes_name.append(current_line)
        current_folder = os.path.join(parent_dir, linea)
    # print(classes_name)

if isdir(model_dir):
    object = {}
    # iterar por el directorio model para buscar subdirectorios
    for subdirectory in os.listdir(model_dir):
        subdir = join(model_dir, subdirectory)
        # iterar solo directorios
        if isdir(subdir):
            # iterar por todos los archivos de texto para conseguir coordenadas
            yolo_line = ""
            for file in os.listdir(subdir):
                filepath = join(subdir, file)
                # ignorar directorios
                if not isdir(filepath):
                    extension = filepath.split(".")[1]
                    # ignorar no textos
                    if extension == "txt":
                        class_name = subdir.split("\\")[-1]
                        index = classes_name.index(class_name)
                        with open(filepath, "r", encoding="utf-8") as file:
                            for linea in file:
                                current_line = linea.strip()
                                # crear linea
                                yolo_line = f"{index} {current_line}"
                # iterar por todos los archivos de img para asignar en el objeto la linea de YOLO si no existe
                for file in os.listdir(subdir):
                    filepath = join(subdir, file)
                    # ignorar directorios
                    if not isdir(filepath):
                        extension = filepath.split(".")[1]
                        # ignorar no textos
                        if extension == "png":
                            img_name = filepath.split("\\")[-1]
                            if img_name in object:
                                if yolo_line not in object[img_name]:
                                    object[img_name].append(yolo_line)
                            else:
                                object[img_name] = [yolo_line]

    # iterar por el objeto creado para crear el archivo de texto con los yolo lines
    label_dir = join(parent_dir, "datasets", "labels", "train")
    for key, values in object.items():
        text_file = join(label_dir, key.replace(".png", ".txt"))
        with open(text_file, "w", encoding="utf-8") as file:
            for value in values:
                file.write(f"{value}\n")